In [16]:
# Import data
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # remove warning

# To show all data in dataframe
pd.set_option('display.max_colwidth', None)

raw = pd.read_json("data/kym.json")                   # 28799 meme templates from Know Your Meme, 16 features
#vision = pd.read_json("kym_vision.json").T         # computer vision data with links to meme instances
#about = pd.read_json("kym_spotlight.json").T       # text entities from the About section on Know Your Meme

In [2]:
# See available options for parameters
# TODO print all options (e.g. types, locations) on screen so that user can know what's possible to select

# Types
types = set()
typedata = raw[raw['details'].map(lambda x: 'type' in x.keys())]['details']
for d in typedata:
    for t in d['type']:
        types.add(t.split("https://knowyourmeme.com/types/")[1])
print("Types available:", types)

# Spread (arbitrarily chosen)
platforms = ["Facebook", "Twitter", "Instagram", "Snapchat", "YouTube", "WhatsApp", "TikTok", "Reddit", "Pinterest", "Tumblr", "LinkedIn", "9GAG", "4chan"]
print("\nPlatforms available:", platforms)

Types available: {'politician', 'music', 'promotion', 'campaign', 'cliche', 'social-media-page', 'slang', 'viral-video', 'company', 'hack', 'dance', 'hashtag', 'performance', 'tabletop-games', 'movement', 'religion', 'organization', 'exploitable', 'application', 'holiday', 'parody', 'social-game', 'emoticon', 'writer', 'catchphrase', 'artist', 'copypasta', 'advertisement', 'competition', 'influencer', 'leak', 'webcomic', 'food', 'podcast', 'theater', 'flash-mob', 'auction', 'law', 'controversy', 'reaction', 'remix', 'protest', 'cartoon', 'prank', 'activist', 'fauna', 'comic-book', 'musician', 'tv-personality', 'programmer', 'sound-effect', 'disaster', 'pop-culture-reference', 'creepypasta', 'visual-effect', 'character', 'video-game', 'album', 'trial', 'businessperson', 'fetish', 'lip-dub', 'viral-debate', 'news-publication', 'axiom', 'reference', 'country', 'filmmaker', 'image-macro', 'gamer', 'fan-labor', 'book', 'marketplace', 'conspiracy-theory', 'product', 'convention', 'participat

In [20]:
# Select parameters for extraction
YEAR_START = 2019  # earliest year is 2010; set to 1 to include all
YEAR_END = 2021
TYPE = ["snowclone", "catchphrase"]  # Meme has at least 1 of the selected types. Leave empty to include all types; keep in mind that roughly half of the memes don't have a type specified.
LOCATION = ["some_region?", "some_language?"]  # TODO
SPREAD = ["facebook", "twitter"]  # "Spread" section of meme contains at least 1 instance of at least 1 of selected keywords. Leave empty to not filter by spread.

In [4]:
# Function definitions

# Matches meme types with specified TYPE. 
# Input: kym dataframe. Output: list of True/False.
def matchType(memes):
    match = []
    for d in memes['details']:
        if 'type' in d.keys():
            for t in d['type']:
                if t.split("https://knowyourmeme.com/types/")[1] in TYPE:
                    match.append(True)
                    break
            else:
                match.append(False)
        else:
            match.append(False)
    return match

# Finds SPREAD keywords in memes' spread data
# Input: kym dataframe. Output: tuple (list of True/False, list of frequency dictionaries of SPREAD keywords).
def matchSpread(memes):
    match = []
    freq_dics = []
    for d in memes['content']:
        if 'spread' in d.keys() and 'text' in d['spread'].keys():
            dic = {keyword: 0 for keyword in SPREAD}
            for text in d['spread']['text']:
                words = text.split(" ")
                for word in words:
                    if word.lower() in SPREAD:
                        dic[word.lower()] += 1
            if sum (dic.values()) == 0:
                match.append(False)
            else:
                match.append(True)
            freq_dics.append(dic)
        else:
            match.append(False)
    return (match, freq_dics)

In [19]:
# Extract data based on parameters
import datetime
from collections import Counter
memes = raw[raw['category'] == "Meme"] # Selecting only memes

# Lower-case all parameters just in case
TYPE = [word.lower() for word in TYPE]
LOCATION = [word.lower() for word in LOCATION]
SPREAD = [word.lower() for word in SPREAD]

# Changing large integers to readable dates
memes['added'] = memes['added'].apply(lambda x: datetime.datetime.fromtimestamp(x) if not pd.isnull(x) else x)
memes['last_update_source'] = memes['last_update_source'].apply(lambda x: datetime.datetime.fromtimestamp(x) if not pd.isnull(x) else x)

years = []
for year in memes['last_update_source']:
    years.append(int(str(year)[:4]) if not pd.isnull(year) else 0)


# Selecting memes that were last updated in YEAR_START - YEAR_END
memes['last_update_source'] = years
memes = memes[memes['last_update_source'] >= YEAR_START]
memes = memes[memes['last_update_source'] <= YEAR_END]

# Removing entries that don't have a description
descriptions = []
missing_desc = []
for title, d in zip(memes['title'], memes['content']):
    if 'about' in d.keys() and 'text' in d['about'].keys():
        descriptions.append(d['about']['text'][0])
    else:
        missing_desc.append(title)
memes = memes[~memes['title'].isin(missing_desc)]
memes['about'] = descriptions

# Selecting memes with at least 1 type in TYPE    
if len(TYPE) > 0:
    memes = memes[matchType(memes)]

    
# Selecting memes that match SPREAD (content:spread contains at least 1 specified keyword)
if len(SPREAD) > 0:
    spread_match, freq_dics = matchSpread(memes)
    memes = memes[spread_match]

# Results
print("Found {} memes matching these criteria.".format(len(memes)))
memes.head()

Found 93 memes matching these criteria.


title  \
96                     Aaaaaaaa   
615   Bitches Love Smiley Faces   
939        First World Problems   
988        First World Problems   
1259       First World Problems   

                                                           url  \
96                     https://knowyourmeme.com/memes/aaaaaaaa   
615   https://knowyourmeme.com/memes/bitches-love-smiley-faces   
939        https://knowyourmeme.com/memes/first-world-problems   
988        https://knowyourmeme.com/memes/first-world-problems   
1259       https://knowyourmeme.com/memes/first-world-problems   

      last_update_source category  \
96                  2021     Meme   
615                 2021     Meme   
939                 2021     Meme   
988                 2021     Meme   
1259                2021     Meme   

                                                         template_image_url  \
96        https://i.kym-cdn.com/entries/icons/original/000/000/993/AAAA.jpg   
615      https://i.kym-cdn.com/entries/icons/original/000/004/286/smily.jpg   
939   https://i.kym-cdn.com/entries/icons/original/000/006/236/1stWorld.jpg   
988   https://i.kym-cdn.com/entries/icons/original/000/006/236/1stWorld.jpg   
1259  https://i.kym-cdn.com/entries/icons/original/000/006/236/1stWorld.jpg   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 meta  \
96    {'og:title': 'Aaaaaaaa', 'og:site_name': 'Know Your Meme', 'og:image': 'https://i.kym-cdn.com/entries/icons/facebook/000/000/993/AAAA.jpg', 'og:image:width': '600', 'og:image:height': '315', 'og:type': 'article', 'fb:app_id': '104675392961482', 'fb:pages': '88519108736', 'article:publisher': 'https://www.facebook.com/knowyourmeme', 'twitter:card': 'summary_large_image', 'twitter:site': '@knowyourmeme', 'twitter:creator': '@knowyourmeme', 'twitter:title': 'Aaaaaaaa', 'twitter:description': 'Aaaaaaaa are a series of image macros and photoshops in which the first letter of the English (or Latin) alphabet “A” is nonsensically repeated many times. These images are often used as either a shock reaction or else in a "shitposting" manner to derail threads. The string of text is also often spammed in chat streams or in text-to-speech engines.', 'twitter:image': 'https://i.kym-cdn.com/entries/icons/facebook/000/000/993/AAAA.jpg', 'description': 'Aaaaaaaa are a series of im

In [186]:
# Question about memes added in recent years

# len(raw[raw['details'].map(lambda x: '2018' in x.values())]) # for some reason, there are no memes after 2017
# raw['added'] = raw['added'].apply(lambda x: datetime.datetime.fromtimestamp(x) if not pd.isnull(x) else x)
# len(raw[raw['added'].map(lambda x: str(x)[:4] == '2021')]) # almost same here, very few images

In [4]:
# Prepare data for Toloka
toloka = memes[['template_image_url', 'about']]
toloka.head()
# toloka.to_csv("toloka.tsv", header = False, index = False, sep="\t")